In [1]:
!python --version
!pip --version

Python 3.10.14
pip 24.0 from /root/home/anaconda3/envs/vllm/lib/python3.10/site-packages/pip (python 3.10)


In [2]:
from vllm import LLM, SamplingParams

# Sample prompts.
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=512)

model_dir = "/mnt/g/code/pretrain_model_dir/_modelscope/qwen/Qwen2-7B-Instruct"
model_dir = "/root/home/my_model/Qwen2-7B-Instruct"
# Create an LLM.
llm = LLM(model=model_dir, max_model_len=1024)  # 如果要用量化, 基本都是需要模型先量化好的
# Generate texts from the prompts. The output is a list of RequestOutput objects
# that contain the prompt, generated text, and other information.
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

WARNING 08-12 22:50:58 config.py:218] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 08-12 22:50:58 llm_engine.py:167] Initializing an LLM engine (v0.5.0.post1) with config: model='/root/home/my_model/Qwen2-7B-Instruct', speculative_config=None, tokenizer='/root/home/my_model/Qwen2-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=/root/home/my_model/Qwen2-7B-Instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 08-12 22:50:58 utils.py:497] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.


ValueError: Cannot find the config file for gptq

In [2]:
tokenizer = llm.get_tokenizer()
tokenizer

CachedQwen2TokenizerFast(name_or_path='/root/home/my_model/Qwen2-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
prompt = "你来自哪里"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
text

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n你来自哪里<|im_end|>\n<|im_start|>assistant\n'

In [8]:
outputs = llm.generate([text], sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r},\n Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.49it/s, est. speed input: 54.81 toks/s, output: 39.86 toks/s]

Prompt: '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n你来自哪里<|im_end|>\n<|im_start|>assistant\n',
 Generated text: '我是一个大模型，没有具体的“出生地”或物理存在。我是'
